In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)

# Input Parameters

Model yang dibuat adalah lapisan bumi homogen secara lateral dengan nilai velocity dan ketebalan masing-masing. Jumlah ray yang digunakan adalah 5.

In [ ]:
rays    = 5 # jumlah ray
velo    = np.array([2300, 2250, 2200, 2150, 2100, 2050, 2000, 1950]) # velocity setiap lapisan
# velo    = np.array([2200, 1300, 2100, 1800, 2500, 1450]) # velocity setiap lapisan
dz      = np.array([100, 100, 100, 100, 100, 100, 100, 100]) # ketebalan lapisan
# dz      = np.array([800, 200, 750, 1200, 500, 250]) # ketebalan lapisan
layers  = len(velo)

In [ ]:
# looping untuk pembentukan sudut takeoff dari setiap ray
# pada kasus ini penulis membuat masing-masing ray memiliki selisih 10 derajat
theta = np.empty(shape = [0, rays])
for i in range(1, rays+1):
    t = i * 10 
    theta = np.array([np.append(theta, t)])

# Ray Parameter

Ketika bekerja pada bidang x dan z, maka komponen vektor slowness arah y ($p_{y}$) bernilai 0 dan menyisakan komponen x ($p_{x}$) dan z ($p_{z}$). Variasi velocity yang digunakan hanya ada pada arah vertikal, sehingga:
$$
p_{x} = p = \sin{i(z)}/V(z)
$$
dimana $i$ adalah sudut takeoff dan $V(z)$ adalah nilai kecepatan pada kedalaman $z$. Dengan menggunakan persamaan eikonal $p^{2} + p_{z}^{2} = V^{-2}$, maka $p_{z}$ dapat dihitung sebagai berikut: 
$$
p_{z} = \sqrt{V^{-2} - p^{2}}
$$


In [ ]:
p = np.empty(shape = [0, rays])
for i in range(rays):
    x = np.sin(np.deg2rad(theta[0, i])) / velo[0] # perhitungan nilai px untuk setiap ray
    p = np.array([np.append(p, x)])

pz = np.empty(shape = [layers, rays])
for j in range(rays):
    for i in range(layers):
        pz[i, j] = np.sqrt((1/(velo[i]))**(2) - (p[0, j])**2) # perhitungan nilai pz setiap ray pada setiap lapisan
print(pz)        

# Lateral Displacement

Perpindahan gelombang secara lateral pada setiap kedalaman dapat dihitung dengan menggunakan rasio antara $p$ dan $p_{z}$ yang sudah diketahui. Dengan demikian:
$$
\frac{dx}{dz} = \frac{p}{p_{z}} = \frac{p}{\sqrt{V^{-2} - p^{2}}}\\
dx = \frac{p}{\sqrt{V^{-2} - p^{2}}} dz
$$

In [ ]:
dx = np.empty(shape = [layers, rays])
for j in range(rays):
    for i in range(layers):
        dx[i, j] = p[0, j] * dz[i]/ pz[i, j] # perpindahan gelombang pada sumbu offset (x) berdasarkan nilai p dan pz
# print(dx)
a = np.array([np.zeros(rays)])
dx = np.append(a, dx, axis = 0)

for j in range(rays):
    for i in range(1, layers + 1):
        dx[i, j] = dx[i, j] + dx[i - 1, j]
# print(dx)

total_depth = np.sum(dz)
dz = np.append(0, dz)
depth = np.array([np.zeros(len(dz))])
for i in range(1, len(dz)):
    depth[0, 0] = total_depth
    depth[0, i] = total_depth - (np.sum(dz[0:i + 1])) 
# print(depth)

# Plotting

In [ ]:
fig, ax = plt.subplots()
for i in range(rays):
    ax.plot(dx[:, i], depth[0, :])
ax.set_title("Ray Tracing")
for i in range(len(depth)):
    plt.hlines(depth[i], 0, np.amax(dx), colors='r')
plt.gca().invert_yaxis()
plt.show()